# Gathering Data

In [1]:
import pandas as pd
import numpy as nm
import matplotlib.pyplot as plt
%matplotlib inline
import requests 
import os
import tweepy
import json
import seaborn as sns
import time

In [2]:
# importing data from the spreadsheet
df1=pd.read_csv('twitter-archive-enhanced.csv')


In [3]:
# downloading the images predictions file 
url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response=requests.get(url)
# saving the file
with open('image-predictions.tsv', mode ='wb') as file:
    file.write(response.content)

In [4]:
# importing the image_prediction.tsv file
df2=pd.read_csv('image-predictions.tsv',sep='\t')

CONSUMER_KEY = 
CONSUMER_SECRET = 
OAUTH_TOKEN =
OAUTH_TOKEN_SECRET =

# using tweepy library

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# list of tweet ids from enhanced data set
df_tweet_ids = df1['tweet_id']

### citation for a GitHub post that helped me in creating the third data frame with data from twitter API.
-Gupta, V. [vaibhavg12]. (2018, September 1). vaibhavg12/udacity-data-wrangling [GitHub post]. GitHub. https://github.com/vaibhavg12/udacity-data-wrangling/blob/master/wrangle_act.ipynb-

# i chaged the format of the below celss to markdown to prevent the code from creating an emty df for the jason as the acsses keys have been deleted from the code .

# creating a list for tweets with exceptions/errors 
tweets_error_list = []

# List of tweets
df_tweets = []

# start time of execution
start = time.time()

# For loop which will add each available tweet json to df_list
for tweet_id in df_tweet_ids:
    try:
        tweet = api.get_status(tweet_id, tweet_mode= 'extended')._json 
        
        favorites = tweet['favorite_count'] # number of favorites for the tweet
        retweets = tweet['retweet_count'] # number of retweets
        user_followers = tweet['user']['followers_count'] # number of followers of the user who tweeted
        user_favourites = tweet['user']['favourites_count'] # number of favourites for the user who tweeted
        date_time = tweet['created_at'] # the timestamp i.e. date and time of creation of the tweet
        
        df_tweets.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets),
                        'user_followers': int(user_followers),
                        'user_favourites': int(user_favourites),
                        'date_time': pd.to_datetime(date_time)})
    except Exception as e:
        print(str(tweet_id)+ " _ " + str(e))
        tweets_error_list.append(tweet_id)
        
# end time for excution
end = time.time()

#printing time for execution
print("Total time taken for execution", end - start)

# total tweets received from api
print("The lengh of the result", len(df_tweets))

# The tweet_id of the errors received from api
print("The lengh of the errors", len(tweets_error_list))

# creating DataFrames
df= pd.DataFrame(df_tweets, columns = ['tweet_id', 'favorites', 'retweets',
                                               'user_followers', 'user_favourites', 'date_time'])
# saving the dataFrame to file
df.to_csv('tweet_json.txt', encoding = 'utf-8', index=False)

In [5]:
# Read the saved tweet_json.txt file into a dataframe
df3 = pd.read_csv('tweet_json.txt', encoding = 'utf-8')

In [6]:
df1 # names and demonators

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


In [7]:
df1.info()
# timestamp type should be date

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [8]:
df1.rating_denominator.value_counts()

10     2333
11        3
50        3
20        2
80        2
0         1
120       1
7         1
170       1
150       1
130       1
90        1
110       1
2         1
70        1
40        1
16        1
15        1
Name: rating_denominator, dtype: int64

In [9]:
df1[df1['rating_denominator'] != 10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" r...","After so many requests, this is Bretagne. She ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023...,204,170,this,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy 4/20 from the squad! 13/10 for all https...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582...,4,20,None,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None


In [10]:
print(df1['text'][1843]) # correct rating
print(df1['text'][1779]) # correct rating
print(df1['text'][1635]) # correct rating
print(df1['text'][1634]) # correct rating
print(df1['text'][1598]) # correct rating
print(df1['text'][1433]) # correct rating
print(df1['text'][1351]) # correct rating
print(df1['text'][1274]) # correct rating
print(df1['text'][1254]) # correct rating
print(df1['text'][1228]) # correct rating
print(df1['text'][902])  # correct rating
print(df1['text'][1120]) # correct rating
print(df1['text'][433]) # correct rating
print(df1['text'][784]) # rating should be 14/10


Here we have an entire platoon of puppers. Total score: 88/80 would pet all at once https://t.co/y93p6FLvVw
IT'S PUPPERGEDDON. Total of 144/120 ...I think https://t.co/ZanVtAtvIq
Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55
Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3
Yes I do realize a rating of 4/20 would've been fitting. However, it would be unjust to give these cooperative pups that low of a rating
Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ
Here is a whole flock of puppers.  60/50 I'll take the lot https://t.co/9dpcw6MdWa
From left to right:
Cletus, Jerome, Alejandro, Burp, &amp; Titson
None know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK
Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.

In [11]:
print(df1['text'][2335]) # rating should be 9/10
print(df1['text'][1662]) # rating should be 10/10
print(df1['text'][1202]) # rating should be 11/10
print(df1['text'][1165]) # rating should be 13/10
print(df1['text'][1068]) # rating should be 14/10
print(df1['text'][313]) # rating should be 13/10

This is an Albanian 3 1/2 legged  Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv
This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5
This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq
Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a
After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ
@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho


In [12]:
print(df1['text'][1663]) # not a rating tweet
print(df1['text'][342]) # not a arting tweet
print(df1['text'][516]) # no rating

I'm aware that I could've said 20/16, but here at WeRateDogs we are very professional. An inconsistent rating scale is simply irresponsible
@docmisterio account started on 11/15/15
Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. 
Keep Sam smiling by clicking and sharing this link:
https://t.co/98tB8y7y7t https://t.co/LouL5vdvxx


In [13]:
df1.name.value_counts() # names a , an , the are wrong 

None        745
a            55
Charlie      12
Lucy         11
Cooper       11
           ... 
Harrison      1
Ziva          1
Major         1
Deacon        1
Holly         1
Name: name, Length: 957, dtype: int64

# inspection 
-print(df1['text'][1663]) # not a rating tweet/ 
 print(df1['text'][342]) # not a arting tweet
 print(df1['text'][516]) # no rating
 
-print(df1['text'][2335]) # rating should be 9/1
print(df1['text'][1662]) # rating should be 10/10
print(df1['text'][1202]) # rating should be 11/10
print(df1['text'][1165]) # rating should be 13/10
print(df1['text'][1068]) # rating should be 14/10
print(df1['text'][313]) # rating should be 13/10

-data type of time stamp coloumn 

In [14]:
df1.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [15]:
df1.sample(6)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
984,749317047558017024,NaN,NaN,2016-07-02 19:01:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Blu. He's a wild bush Floofer. I wish ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749317047...,12,10,Blu,None,floofer,None,None
341,832215726631055365,NaN,NaN,2017-02-16 13:11:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Moreton. He's the Good ...,7.932865e+17,4.196984e+09,2016-11-01 03:00:09 +0000,https://twitter.com/dog_rates/status/793286476...,13,10,Moreton,None,None,None,None
1353,703769065844768768,NaN,NaN,2016-02-28 02:29:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you're trying to watch your favorite tv s...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/703769065...,10,10,None,None,None,None,None
547,805520635690676224,NaN,NaN,2016-12-04 21:14:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zeke the Wonder Dog. He never let that...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/805520635...,13,10,Zeke,None,None,None,None
1934,674014384960745472,NaN,NaN,2015-12-07 23:55:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Aiden. His eyes are magical. Love...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/674014384...,11,10,Aiden,None,None,None,None
797,773191612633579521,NaN,NaN,2016-09-06 16:10:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Grey. He's the dogtor in charge of you...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/773191612...,12,10,Grey,None,None,None,None


In [16]:
df1.tweet_id.duplicated().sum().sum()

0

In [17]:
df2

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [18]:
df2.info() # tweet_id should be a str 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [19]:
df2.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [20]:
df2.jpg_url.duplicated().sum().sum()

66

In [21]:
df3

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
0,892420643555336193,34985,7341,8983857,145822,2017-08-01 16:23:56+00:00
1,892177421306343426,30292,5477,8983857,145822,2017-08-01 00:17:27+00:00
2,891815181378084864,22789,3621,8983857,145822,2017-07-31 00:18:03+00:00
3,891689557279858688,38248,7530,8983857,145822,2017-07-30 15:58:51+00:00
4,891327558926688256,36527,8109,8983857,145822,2017-07-29 16:00:24+00:00
...,...,...,...,...,...,...
2326,666049248165822465,93,38,8983898,145822,2015-11-16 00:24:50+00:00
2327,666044226329800704,259,122,8983898,145822,2015-11-16 00:04:52+00:00
2328,666033412701032449,107,39,8983898,145822,2015-11-15 23:21:54+00:00
2329,666029285002620928,117,41,8983898,145822,2015-11-15 23:05:30+00:00


In [22]:
df3 # user_favourites data seem wrong

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
0,892420643555336193,34985,7341,8983857,145822,2017-08-01 16:23:56+00:00
1,892177421306343426,30292,5477,8983857,145822,2017-08-01 00:17:27+00:00
2,891815181378084864,22789,3621,8983857,145822,2017-07-31 00:18:03+00:00
3,891689557279858688,38248,7530,8983857,145822,2017-07-30 15:58:51+00:00
4,891327558926688256,36527,8109,8983857,145822,2017-07-29 16:00:24+00:00
...,...,...,...,...,...,...
2326,666049248165822465,93,38,8983898,145822,2015-11-16 00:24:50+00:00
2327,666044226329800704,259,122,8983898,145822,2015-11-16 00:04:52+00:00
2328,666033412701032449,107,39,8983898,145822,2015-11-15 23:21:54+00:00
2329,666029285002620928,117,41,8983898,145822,2015-11-15 23:05:30+00:00


In [23]:
df3.user_favourites.value_counts() # most of the data in the user_favourites coloumn are the same which indicates it is wrong

145822    2314
145817      15
145815       2
Name: user_favourites, dtype: int64

In [24]:
df3.info() #date_time data is str.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tweet_id         2331 non-null   int64 
 1   favorites        2331 non-null   int64 
 2   retweets         2331 non-null   int64 
 3   user_followers   2331 non-null   int64 
 4   user_favourites  2331 non-null   int64 
 5   date_time        2331 non-null   object
dtypes: int64(5), object(1)
memory usage: 109.4+ KB


In [25]:
df3.tweet_id.duplicated().sum()

0

In [26]:
df3.sample(5)

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
1913,673906403526995968,2963,1517,8983888,145822,2015-12-07 16:46:21+00:00
597,796563435802726400,0,7072,8983871,145822,2016-11-10 04:01:37+00:00
574,798933969379225600,12889,4368,8983871,145822,2016-11-16 17:01:16+00:00
1963,672834301050937345,1214,526,8983889,145822,2015-12-04 17:46:12+00:00
1301,706166467411222528,4947,1523,8983878,145822,2016-03-05 17:16:20+00:00


In [27]:
df3.user_followers.describe() # values are too close and seem inaccurte 

count    2.331000e+03
mean     8.983881e+06
std      4.540861e+01
min      8.983857e+06
25%      8.983871e+06
50%      8.983875e+06
75%      8.983887e+06
max      8.984409e+06
Name: user_followers, dtype: float64

# Cleaning

In [28]:
tweets=df1.copy()
images=df2.copy()
tweets_json=df3.copy()

# for the tweets df 
## Quality 
1- change data type of timestamp to datetime 

2- delete rows with no ratings

3-adjust rating_numerator and rating_denominator couloums

4-change a ,an , the in the name couloumn to none

5-filter nan values in the retweeted_status_id coloumn 

6-extract the tweet source from the source coloumn 

7-the tweet_id coloumn name should be'tweet_id' in all dataframes and it's data type should be string.

## tidiness

1- delete unused coloumns 

2- melt doggo	floofer	pupper	puppo coloumns

# for the imaages df

## Quality

1-the tweet_id coloumn name should be'tweet_id' in all dataframes and it's data type should be string.

2-drop tweets with duplicated dog images.

## tidiness

1- make one coloumn from prediction and one coloumn for conf_lvl

# for tweets_json df

## Quality

1-the tweet_id coloumn name should be'tweet_id' in all dataframes and it's data type should be string.

2-change date_time data type to datetime.

3-drop the user_favourites and ,user_followers coloumns 



# Quality 
1- change data type of timestamp to datetime

In [29]:
tweets.timestamp=pd.to_datetime(tweets.timestamp)
# test 
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   int64              
 1   in_reply_to_status_id       78 non-null     float64            
 2   in_reply_to_user_id         78 non-null     float64            
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    float64            
 7   retweeted_status_user_id    181 non-null    float64            
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

2- delete rows with no ratings

In [30]:
# get the tweet_id 
del_id=[df1['tweet_id'][1663],df1['tweet_id'][342],df1['tweet_id'][516]]


In [31]:
# delete the 3 rows 
tweets=tweets[tweets['tweet_id'] != del_id[0]]
tweets=tweets[tweets['tweet_id'] != del_id[1]]
tweets=tweets[tweets['tweet_id'] != del_id[2]]

In [32]:
# test 
tweets[tweets['tweet_id'] == del_id[0]]
tweets[tweets['tweet_id'] == del_id[1]]
tweets[tweets['tweet_id'] == del_id[2]]


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


3-adjust rating_numerator and rating_denominator couloums

df1['text'][2335] # rating should be 9/10

df1['text'][1662] # rating should be 10/10

df1['text'][1202] # rating should be 11/10

df1['text'][1165] # rating should be 13/10

df1['text'][1068] # rating should be 14/10

df1['text'][313] # rating should be 13/10

In [33]:
# getting tweer_ids
adj_id=[df1['tweet_id'][2335],df1['tweet_id'][1662],df1['tweet_id'][1202],df1['tweet_id'][1165],df1['tweet_id'][1068],df1['tweet_id'][313]]


In [34]:
# setting all dumenators to 10 
for i in adj_id :
    tweets.loc[(tweets.tweet_id == i), 'rating_denominator'] = 10
# test 
tweets.loc[(tweets.tweet_id == adj_id[3]), 'rating_denominator'] 

1165    10
Name: rating_denominator, dtype: int64

In [35]:
#correnting numenators manually 
tweets.loc[(tweets.tweet_id == adj_id[0]), 'rating_numerator'] = 9
tweets.loc[(tweets.tweet_id == adj_id[1]), 'rating_numerator'] = 10
tweets.loc[(tweets.tweet_id == adj_id[2]), 'rating_numerator'] = 11
tweets.loc[(tweets.tweet_id == adj_id[3]), 'rating_numerator'] = 13
tweets.loc[(tweets.tweet_id == adj_id[4]), 'rating_numerator'] = 14
tweets.loc[(tweets.tweet_id == adj_id[5]), 'rating_numerator'] = 13

In [36]:
# test 
tweets[tweets['rating_denominator'] != 10]


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40+00:00,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06+00:00,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023...,204,170,this,None,None,None,None
1228,713900603437621249,NaN,NaN,2016-03-27 01:29:02+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday here's 9 puppers on a bench. 99...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713900603...,99,90,None,None,None,None,None
1254,710658690886586372,NaN,NaN,2016-03-18 02:46:49+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Here's a brigade of puppers. All look very pre...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/710658690...,80,80,None,None,None,None,None
1274,709198395643068416,NaN,NaN,2016-03-14 02:04:08+00:00,"<a href=""http://twitter.com/download/iphone"" r...","From left to right:\nCletus, Jerome, Alejandro...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/709198395...,45,50,None,None,None,None,None
1351,704054845121142784,NaN,NaN,2016-02-28 21:25:30+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Here is a whole flock of puppers. 60/50 I'll ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704054845...,60,50,a,None,None,None,None
1433,697463031882764288,NaN,NaN,2016-02-10 16:51:59+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Happy Wednesday here's a bucket of pups. 44/40...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/697463031...,44,40,None,None,None,None,None
1598,686035780142297088,6.860340e+17,4.196984e+09,2016-01-10 04:04:10+00:00,"<a href=""http://twitter.com/download/iphone"" r...",Yes I do realize a rating of 4/20 would've bee...,NaN,NaN,NaN,NaN,4,20,None,None,None,None,None


4-change a ,an , the in the name couloumn to none

In [37]:
# code
tweets.name=tweets.name.replace('a','none')
tweets.name=tweets.name.replace('an','none')
tweets.name=tweets.name.replace('the','none')

In [38]:
# test
tweets[tweets.name == 'a']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [39]:
# test
tweets[tweets.name == 'an']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [40]:
# test
tweets[tweets.name == 'the']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


5-filter nan values in the retweeted_status_id coloumn


In [41]:
#codep
tweets=tweets[pd.isnull(tweets. retweeted_status_id)]

In [42]:
# test 
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2172 non-null   int64              
 1   in_reply_to_status_id       76 non-null     float64            
 2   in_reply_to_user_id         76 non-null     float64            
 3   timestamp                   2172 non-null   datetime64[ns, UTC]
 4   source                      2172 non-null   object             
 5   text                        2172 non-null   object             
 6   retweeted_status_id         0 non-null      float64            
 7   retweeted_status_user_id    0 non-null      float64            
 8   retweeted_status_timestamp  0 non-null      object             
 9   expanded_urls               2116 non-null   object             
 10  rating_numerator            2172 non-null   int64           

6-extract the tweet source from the source coloumn

In [43]:
# code
source=[]
for i in tweets.source :
    source.append(i.split(' ')[-3][-7:]+' '+i.split(' ')[-2]+' '+i.split(' ')[-1][:-4])
tweets.source=source

In [44]:
# test 
tweets.source.value_counts()

Twitter for iPhone                                                                 2039
Make a Scene                                                                         91
Twitter Web Client                                                                   31
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck      11
Name: source, dtype: int64

In [45]:
tweets[tweets.source=='Make a Scene']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
529,808344865868283904,NaN,NaN,2016-12-12 16:16:49+00:00,Make a Scene,This is Seamus. He's very bad at entering pool...,NaN,NaN,NaN,https://vine.co/v/5QWd3LZqXxd,11,10,Seamus,None,None,None,None
562,802600418706604034,NaN,NaN,2016-11-26 19:50:26+00:00,Make a Scene,This is Bailey. She has mastered the head tilt...,NaN,NaN,NaN,https://vine.co/v/5FwUWjYaW0Y,11,10,Bailey,None,None,None,None
657,791774931465953280,NaN,NaN,2016-10-27 22:53:48+00:00,Make a Scene,Vine will be deeply missed. This was by far my...,NaN,NaN,NaN,https://vine.co/v/ea0OwvPTx9l,14,10,None,None,None,None,None
672,789903600034189313,NaN,NaN,2016-10-22 18:57:48+00:00,Make a Scene,This is Ralphy. His dreams were just shattered...,NaN,NaN,NaN,https://vine.co/v/5wPT1aBxPQZ,13,10,Ralphy,None,None,pupper,None
699,786286427768250368,NaN,NaN,2016-10-12 19:24:27+00:00,Make a Scene,This is Arnie. He's afraid of his own bark. 12...,NaN,NaN,NaN,https://vine.co/v/5XH0WqHwiFp,12,10,Arnie,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,676916996760600576,NaN,NaN,2015-12-16 00:09:23+00:00,Make a Scene,Super speedy pupper. Does not go gentle into t...,NaN,NaN,NaN,https://vine.co/v/imJ0BdZOJTw,10,10,None,None,None,pupper,None
1818,676593408224403456,NaN,NaN,2015-12-15 02:43:33+00:00,Make a Scene,This pupper loves leaves. 11/10 for committed ...,NaN,NaN,NaN,https://vine.co/v/eEQQaPFbgOY,11,10,None,None,None,pupper,None
1834,676121918416756736,NaN,NaN,2015-12-13 19:30:01+00:00,Make a Scene,Here we are witnessing a very excited dog. Cle...,NaN,NaN,NaN,https://vine.co/v/iZXg7VpeDAv,8,10,None,None,None,None,None
1916,674307341513269249,NaN,NaN,2015-12-08 19:19:32+00:00,Make a Scene,This is life-changing. 12/10 https://t.co/SroT...,NaN,NaN,NaN,https://vine.co/v/i7nWzrenw5h,12,10,life,None,None,None,None


In [46]:
dft=df1[df1.tweet_id== 808344865868283904]
print(dft['source'][529])

<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>


In [47]:
#needs more adjustment 
tweets.source=tweets.source.replace('<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck','TweetDeck')
tweets.source=tweets.source.replace('Make a Scene','vine.co')

In [48]:
#test 
tweets.source.value_counts()

Twitter for iPhone    2039
vine.co                 91
Twitter Web Client      31
TweetDeck               11
Name: source, dtype: int64

7-the tweet_id coloumn name should be'tweet_id' in all dataframes and it's data type should be string.

In [49]:
# code
tweets.tweet_id=tweets.tweet_id.astype(str)

In [50]:
#test 
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2172 non-null   object             
 1   in_reply_to_status_id       76 non-null     float64            
 2   in_reply_to_user_id         76 non-null     float64            
 3   timestamp                   2172 non-null   datetime64[ns, UTC]
 4   source                      2172 non-null   object             
 5   text                        2172 non-null   object             
 6   retweeted_status_id         0 non-null      float64            
 7   retweeted_status_user_id    0 non-null      float64            
 8   retweeted_status_timestamp  0 non-null      object             
 9   expanded_urls               2116 non-null   object             
 10  rating_numerator            2172 non-null   int64           

# tidiness
1- delete unused coloumns

In [51]:
# code
col=['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','expanded_urls']
tweets=tweets.drop(col,1)

In [52]:
# test 
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2172 non-null   object             
 1   timestamp           2172 non-null   datetime64[ns, UTC]
 2   source              2172 non-null   object             
 3   text                2172 non-null   object             
 4   rating_numerator    2172 non-null   int64              
 5   rating_denominator  2172 non-null   int64              
 6   name                2172 non-null   object             
 7   doggo               2172 non-null   object             
 8   floofer             2172 non-null   object             
 9   pupper              2172 non-null   object             
 10  puppo               2172 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(2), object(8)
memory usage: 203.6+ KB


2- melt doggo floofer pupper puppo coloumns

In [53]:
#code
tweets=pd.melt(tweets, id_vars=['tweet_id','timestamp','text','rating_numerator','rating_denominator','name'],value_name='dog_type',var_name='drop')

In [54]:
tweets=tweets.drop('drop',1)

In [55]:
# test
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10860 entries, 0 to 10859
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            10860 non-null  object             
 1   timestamp           10860 non-null  datetime64[ns, UTC]
 2   text                10860 non-null  object             
 3   rating_numerator    10860 non-null  int64              
 4   rating_denominator  10860 non-null  int64              
 5   name                10860 non-null  object             
 6   dog_type            10860 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(2), object(4)
memory usage: 594.0+ KB


In [56]:
tweets.dog_type.value_counts()

None                  8332
Twitter for iPhone    2039
pupper                 234
vine.co                 91
doggo                   87
Twitter Web Client      31
puppo                   25
TweetDeck               11
floofer                 10
Name: dog_type, dtype: int64

# for the imaages df
## Quality
1-the tweet_id coloumn name should be'tweet_id' in all dataframes and it's data type should be string.

2-drop tweets with duplicated dog images.

## tidiness
1- make one coloumn from prediction and one coloumn for conf_lvl

## Quality
1-the tweet_id coloumn name should be'tweet_id' in all dataframes and it's data type should be string.

In [57]:
#code 
images.tweet_id=images.tweet_id.astype(str)
# test
images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   object 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


2-drop tweets with duplicated dog images.

In [58]:
#code 
images=images.drop_duplicates(subset=['jpg_url'])

In [59]:
images.jpg_url.duplicated().sum()

0

## tidiness
1- make one coloumn from prediction and one coloumn for conf_lvl

In [60]:
images.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [61]:
images.p3_dog.value_counts()

True     1450
False     559
Name: p3_dog, dtype: int64

In [62]:
images.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2009 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2009 non-null   object 
 1   jpg_url   2009 non-null   object 
 2   img_num   2009 non-null   int64  
 3   p1        2009 non-null   object 
 4   p1_conf   2009 non-null   float64
 5   p1_dog    2009 non-null   bool   
 6   p2        2009 non-null   object 
 7   p2_conf   2009 non-null   float64
 8   p2_dog    2009 non-null   bool   
 9   p3        2009 non-null   object 
 10  p3_conf   2009 non-null   float64
 11  p3_dog    2009 non-null   bool   
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 162.8+ KB


In [ ]:
#code
breed=[]
cf=[]
for i in images.tweet_id :
    if images.loc[(images.tweet_id == i),'p1_dog'].iloc[0] == True:
        breed.append(images.loc[(images.tweet_id == i),'p1'].iloc[0])
        cf.append(images.loc[(images.tweet_id == i),'p1_conf'].iloc[0])
    elif images.loc[(images.tweet_id == i),'p2_dog'].iloc[0] == True:
        breed.append(images.loc[(images.tweet_id == i),'p2'].iloc[0])
        cf.append(images.loc[(images.tweet_id == i),'p2_conf'].iloc[0])
    elif images.loc[(images.tweet_id == i),'p3_dog'].iloc[0] == True:
        breed.append(images.loc[(images.tweet_id == i),'p3'].iloc[0])
        cf.append(images.loc[(images.tweet_id == i),'p3_conf'].iloc[0])
    else :
        breed.append('NA')
        cf.append('0')
    
images['breed']=breed
images['conf_lvl']=cf
images=images.drop(['p1','p1_conf','p1_dog','p2','p2_conf','p2_dog','p3','p3_conf','p3_dog'],1)

In [ ]:
# remove rows with no right prediction
images=images[images.breed !='NA']

In [ ]:
# change data type of conf_lvl to float
images.conf_lvl=images.conf_lvl.astype(float)

In [ ]:
# test
images.head()

In [ ]:
images[images.breed =='NA']


In [ ]:
images.info()

# for tweets_json df
## Quality
1-the tweet_id coloumn name should be'tweet_id' in all dataframes and it's data type should be string.

2-change date_time data type to datetype

3-drop the user_favourites and ,user_followers coloumns

## Quality
1-the tweet_id coloumn name should be'tweet_id' in all dataframes and it's data type should be string.

In [ ]:
# code
tweets_json.tweet_id=tweets_json.tweet_id.astype(str)

In [ ]:
# test 
tweets_json.info()

2-change date_time data type to datetype

In [ ]:
# code
tweets_json.date_time=pd.to_datetime(tweets_json.date_time)

In [ ]:
# test 
tweets_json.info()

3-drop the user_favourites and ,user_followers coloumns

In [ ]:
# code 
tweets_json=tweets_json.drop(['user_favourites','user_followers'], 1)

In [ ]:
# test 
tweets_json.head()

## merging all data into a single df

In [ ]:
# merge the data into one data frame
df_twitter1 = pd.merge(tweets,images,how = 'left', on = ['tweet_id'])aa
df_twitter = pd.merge(df_twitter1,tweets_json,how = 'left', on = ['tweet_id'])

inscpect data to make sure no further cleaning is needed

In [ ]:
df_twitter.info() 

In [ ]:
df_twitter.head() # timestamp and date_time coloumn are the same , some tweets have no image 


In [ ]:
# keep only non nan jpg_url data 
df_twitter=df_twitter[df_twitter['jpg_url'].notnull()]

In [ ]:
# test
df_twitter.info()

In [ ]:
# drop the timestamp coloumn
df_twitter=df_twitter.drop(['timestamp'],1)


In [ ]:
#test 
df_twitter.head()

In [ ]:
#Store the clean DataFrame in a CSV file
df_twitter.to_csv('twitter_archive_master.csv',index=False, encoding = 'utf-8')

# insights and vizualisation

## getting the most owned dog breed in the data set

In [ ]:
df_twitter.breed.value_counts()

In [ ]:
dog_breed=df_twitter.groupby('breed').filter(lambda x: len(x) >=200)
dog_breed.breed.value_counts()
names=['golden_retriever','Labrador_retriever','Pembroke','pug','toy_poodle','chow','Samoyed','Pomeranian','malamute','French_bulldog','Chesapeake_Bay_retriever','cocker_spaniel','miniature_pinscher']

In [ ]:
dog_breed.breed.value_counts()

In [ ]:
names=dog_breed.breed.value_counts().index.tolist()
counts=list(dog_breed.breed.value_counts())

In [ ]:
counts

In [ ]:
names

### golden retriever is the most common type dog 

In [ ]:
# pie chart to show most popular dog type
figure1, axis1 = plt.subplots()
axis1.pie(counts,labels = names, shadow = True, startangle = 90);


# geting the dog breed with the highest rate

In [ ]:
dog_breed.info()

In [ ]:
# for dogs breeds that are mentioned more than 100 times 
dog_breed['rate']=dog_breed.rating_numerator /dog_breed.rating_denominator

In [ ]:
dog_breed.groupby('breed').mean()['rate']

## the hightes rated dog breed is golden_retriever

# dog breed with the highest confident level in predicting 

In [ ]:
# change data type of conf_lvl to float
df_twitter.conf_lvl=df_twitter.conf_lvl.astype(float)

In [ ]:
df_twitter.info()

In [ ]:
dog_breed.groupby('breed').mean()['conf_lvl']

## the dog with the highest confident level in image prediction is Samoyed